https://github.com/fmfn/BayesianOptimization

In [1]:
#read dfs# import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
train_X=pd.read_csv("../input/vol-1/train_x.csv")
train_y=pd.read_csv("../input/vol-1/train_y.csv")
normalized_test_df=pd.read_csv("../input/vol-1/normalized_test_df.csv")
df_test1=pd.read_csv("../input/vol-1/df_test1.csv")

In [2]:
train_X=train_X.drop(columns=train_X.columns[0])
train_y=train_y.drop(columns=train_y.columns[0])
normalized_test_df=normalized_test_df.drop(columns=normalized_test_df.columns[0])
df_test1=df_test1.drop(columns=df_test1.columns[0])

In [3]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error

def xgboostcv(max_depth,
              learning_rate,
              n_estimators,
              gamma,
              min_child_weight,
              max_delta_step,
              subsample,
              reg_lambda,
              colsample_bytree,
              verbosity=0,
              nthread=-1):
    
    model=xgb.XGBRegressor(max_depth=int(round(max_depth)),
                           learning_rate=learning_rate,
                           n_estimators=int(round(n_estimators)),
                           verbosity=verbosity,
                           nthread=nthread,
                           gamma=gamma,
                           min_child_weight=min_child_weight,
                           max_delta_step=int(round(max_delta_step)),
                           subsample=subsample,
                           reg_lambda=reg_lambda,
                           colsample_bytree=colsample_bytree,
                           tree_method='gpu_hist')
    
    result = cross_val_score(model,train_X,train_y,scoring="neg_root_mean_squared_error",cv=5).mean()
    
    return result


# Load data set and target values
xgboostBO = BayesianOptimization(f = xgboostcv,
                                 pbounds = {'max_depth': (5, 10),
                                          'learning_rate': (0.01, 0.3),
                                          'n_estimators': (50, 1000),
                                          'gamma': (1., 0.01),
                                          'min_child_weight': (2, 10),
                                          'max_delta_step': (0, 0.1),
                                          'subsample': (0.6, 0.8),
                                          'reg_lambda': (0, 1.),
                                          'colsample_bytree' :(0.5, 0.99)
                                          },
                                random_state=1)

xgboostBO.maximize(init_points=2, n_iter=4)
print('-'*53)

print('Final Results')
for res in xgboostBO.res:
    print('XGBOOST: %f' % res['target'])

print('*'*53)
print('best params ==>', xgboostBO.max)

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | n_esti... | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        | -1.999e+0 |  0.7043   |  0.2869   |  0.01003  |  0.03023  |  5.734    |  2.739    |  226.9    |  0.3456   |  0.6794   |
|  2        | -2.114e+0 |  0.764    |  0.585    |  0.2087   |  0.02045  |  9.391    |  2.219    |  686.9    |  0.4173   |  0.7117   |
|  3        | -1.943e+0 |  0.7353   |  0.01     |  0.1212   |  0.03918  |  5.799    |  3.374    |  227.9    |  0.7382   |  0.7008   |
|  4        | -1.955e+0 |  0.55     |  0.01     |  0.2342   |  0.04911  |  5.075    |  5.025    |  228.4    |  0.7212   |  0.7864   |
|  5        | -1.965e+0 |  0.5762   |  0.01     |  0.2059   |  0.03942  |  6.662    |  3.562    |  231.0    |  0.2053   |  0.7555   |
|  6        | -1.944e+0 |  0.8      |  0.01     |  0.07018  | 

In [4]:
param_dict = xgboostBO.max['params'] 
param_dict['tree_method']='gpu_hist'

# we need to round some float values to integer since these parameters works only in integer
param_dict['max_depth']=int(round(param_dict['max_depth']))
param_dict['n_estimators']=int(round(param_dict['n_estimators']))
param_dict['max_delta_step']=int(round(param_dict['max_delta_step']))


In [5]:
best_xgb_model = xgb.XGBRegressor()
best_xgb_model.set_params(**param_dict)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7352867358935102,
             gamma=0.01, gpu_id=None, importance_type='gain',
             interaction_constraints=None, learning_rate=0.12115604078643202,
             max_delta_step=0, max_depth=6, min_child_weight=3.3739655014413676,
             missing=nan, monotone_constraints=None, n_estimators=228,
             n_jobs=None, num_parallel_tree=None, random_state=None,
             reg_alpha=None, reg_lambda=0.7381672453007786,
             scale_pos_weight=None, subsample=0.7007545779375324,
             tree_method='gpu_hist', validate_parameters=None, verbosity=None)

In [6]:
best_xgb_model.fit(train_X,train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7352867358935102,
             gamma=0.01, gpu_id=0, importance_type='gain',
             interaction_constraints='', learning_rate=0.12115604078643202,
             max_delta_step=0, max_depth=6, min_child_weight=3.3739655014413676,
             missing=nan,
             monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
             n_estimators=228, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=0.7381672453007786, scale_pos_weight=1,
             subsample=0.7007545779375324, tree_method='gpu_hist',
             validate_parameters=1, verbosity=None)

In [7]:
y_pred = best_xgb_model.predict(normalized_test_df)

In [8]:
# prediction results
pred_df=pd.DataFrame(zip(df_test1['sector_id'], y_pred), columns=['sector_id', 'predictions'])
submission_dff=pred_df.groupby('sector_id').sum()
ss=submission_dff.reset_index()

ss=ss.rename(columns={"sector_id": "Sektor_Id", "predictions": "Expected"})

In [9]:
ss.to_csv('out_bayesian.csv', index=False)